# 프로젝트: ResNet Ablation Study

## 1) ResNet 기본 블록 구성하기

![resnetstruc](../image/deeper2ex_resnet.png)

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras import layers

In [2]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [3]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [4]:
import urllib3
urllib3.disable_warnings()

ds_train, ds_info = tfds.load(
    'cifar10',
    split='train[:1%]',
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)


In [5]:
print(ds_info.features)

FeaturesDict({
    'id': Text(shape=(), dtype=tf.string),
    'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})


In [6]:
# 데이터의 개수도 확인해 봅시다. 
print(tf.data.experimental.cardinality(ds_train))

tf.Tensor(500, shape=(), dtype=int64)


In [7]:
# ds_train = apply_normalize_on_dataset(ds_train, batch_size=100)

## 2) ResNet-34, ResNet-50 Complete Model

ResNet-34  
VGG와 같이 블록을 만드는 함수를 사용해서 직접 전체 모델을 만들어 봅시다.  
ResNet-34와 ResNet-50의 차이에 따라 달라지는 구성(configuration)을 함수에 전달해서  
같은 생성 함수 build_resnet()를 통해서 ResNet의 여러가지 버전들을 모두 만들어 낼 수 있도록 해야 합니다.

In [8]:
cha = [31,12]

In [9]:
cha[1]

12

In [10]:
# resnet블록 생성
def build_resnet_block(input_layer=(64,64,3), kernel_size=(3,3), strides=1, filters=[64,64]):
    # 입력 레이어
    shortcut = x = input_layer # 64,64,3
    #cnn1
    print(x)
    shortcut= x = keras.layers.Conv2D(
        filters=filters[0],
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        
        padding='same'
        )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    #cnn2
    x = keras.layers.Conv2D(
        filters=filters[1],
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        
        padding='same'
        )(x)
    x = keras.layers.BatchNormalization()(x)
    print(x.shape)
    x = keras.layers.Add()([x,shortcut])
    x = keras.layers.Activation('relu')(x)
    
    return x
    
    
    

In [11]:
def dot_residual(input_layer=(64,64,3), kernel_size=(3,3), strides=1, filters=[64,64]):
    # 입력 레이어
    shortcut = x = input_layer # 64,64,3
    #cnn1
    print(x)
    shortcut= x = keras.layers.Conv2D(
        filters=2,
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        
        padding='same'
        )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    #cnn2
    x = keras.layers.Conv2D(
        filters=1,
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        
        padding='same'
        )(x)
    x = keras.layers.BatchNormalization()(x)
    print(x.shape)
    shortcut = keras.layers.Conv2D(
        filters=1,
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        padding='same'
        )(shortcut)
    shortcut = keras.layers.BatchNormalization()(shortcut)
            
    x = keras.layers.Add()([x,shortcut])
    x = keras.layers.Activation('relu')(x)
    
    return x

In [12]:
input_layer = keras.layers.Input(shape=(64,64,3))

In [13]:
build_resnet = build_resnet_block(kernel_size=(3,3), strides=1, input_layer=input_layer)

Tensor("input_1:0", shape=(None, 64, 64, 3), dtype=float32)
(None, 64, 64, 64)


In [14]:
model = keras.Model(inputs=input_layer, outputs=build_resnet) 

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [16]:
def ResNet34(input_shape=(32,32,3),num_classes=10):
    x = keras.layers.Input(input_shape)
    x = keras.layers.Conv2D(64, (7,7), padding='same', strides=2, activation='relu')(x)
    x = keras.layers.MaxPool2D((3,3), padding='same', strides=2)(x)
    # block 1-3(with 2)
    for i in range(3):
        x = build_resnet_block(input_layer=x, kernel_size=(3,3), strides=1, filters = [64,64])(x)
    return x
    
    # /2, block4-6
    x = dot_residual(input_layer=(64,64,3), kernel_size=(3,3), strides=1, filters=[128,128])(x)
    for i in range(3):
        x = build_resnet_block(input_layer=x, kernel_size=(3,3), strides=1, filters = [128,128])(x)
    return x

    x = dot_residual(input_layer=(64,64,3), kernel_size=(3,3), strides=1, filters=[256,256])(x)
    for i in range(3):
        x = build_resnet_block(input_layer=x, kernel_size=(3,3), strides=1, filters = [256,256])(x)
    return x

    x = dot_residual(input_layer=(64,64,3), kernel_size=(3,3), strides=1, filters=[512,512])(x)
    
    for i in range(2):
        x = build_resnet_block(input_layer=x, kernel_size=(3,3), strides=1, filters = [512,512])(x)
    x = keras.layers.AveragePooling2D()(x)
    x = keras.layers.Flatten()
    x = keras.layers.Dense(num_classes, activation='softmax')(x)
    
    
    
    

In [17]:
Res = ResNet34()

Tensor("max_pooling2d/Identity:0", shape=(None, 8, 8, 64), dtype=float32)
(None, 8, 8, 64)


TypeError: 'Tensor' object is not callable

In [ ]:
model1 = keras.Model(inputs=input_layer, outputs=Res) 

In [ ]:
def bottleneck(input_layer=(64,64,3), kernel_size=(3,3), strides=1, filters=[64,64,256]):
    # 입력 레이어
    shortcut= x = input_layer # 64,64,3
    # 1*1
    x = keras.layers.Conv2D(
        filters=filters[0],
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        padding='same'
        )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    # filter 
    x = keras.layers.Conv2D(
        filters=filters[1],
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        padding='same'
        )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    # 1*1
    x = keras.layers.Conv2D(
        filters=filters[2],
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        padding='same'
        )(x)
    # skip
    shortcut = keras.layers.Conv2D(
        filters=1,
        kernel_size=kernel_size,
        kernel_initializer='he_normal',
        padding='same'
        )(shortcut)
    shortcut = keras.layers.BatchNormalization()(shortcut)
            
    x = keras.layers.Add()([x,shortcut])
    x = keras.layers.Activation('relu')(x)
    
    return x

In [ ]:
def resnet50(input_shape=(32,32,3),num_classes=10):
    x = keras.layers.Input(input_shape)
    x = keras.layers.Conv2D(64, (7,7), padding='same', strides=2, activation='relu')(x)
    x = keras.layers.MaxPool2D((3,3), padding='same', strides=2)(x)
    
    for i in range(3):
        bottleneck(input_layer= x, kernel_size=(3,3), strides=1, filters=[64,64,256])(x)
    return x

    x = keras.layers.Conv2D(128 ,(3,3), padding='same', strides=2, activation='relu')(x)
    x = keras.layers.Conv2D(128 ,(3,3), padding='same', strides=1, activation='relu')(x)
    for i in range(3):
        bottleneck(input_layer= x, kernel_size=(3,3), strides=1, filters=[256,256,1024])(x)
    return x
        
    

    